# Project: Speech Emotional Recognition

## Requirements
1. OpenSmile

Assuming all required files are within the "data" directory

In [121]:
data_path = "/work/peiyun/data"

## 1. Write utterance labels to CSV files

Import the required modules.

In [122]:
import csv
import os

Write the label csv file.

In [123]:
with open(os.path.join(data_path, "label.csv") , mode='w') as label_file:
    writer = csv.writer(label_file, delimiter=",")
    
    for session in range(1,6):
        path = os.path.join(data_path, "Session" + str(session), "dialog", "EmoEvaluation")
        
        for filename in os.listdir(path):  # file in each session
            
            # for summary txt files
            if filename.endswith(".txt"):
                f = open(os.path.join(path, filename), "r")
                for line in f.readlines():
                    if line[0] == "[":
                        # extract name and label of the instance and write to csv
                        writer.writerow(line.split("\t")[1:3])
                f.close()
                
    label_file.close()

## 2. Feature Extraction with openSMILE

File for feature extractions.

In [136]:
%%file feature.py

data_path = "/work/peiyun/data"

# Import the required modules
import argparse
import os
from subprocess import call
import csv
import sys

# Get the ground_truth label of the file
def get_label(label_file, filename):
    with open(label_file, mode = "r") as f:
        reader = csv.reader(f)
        for row in reader:
            name, label = row
            if name == filename.strip(".wav"):
                return label

# Check input and output directories
def check_dirs(args):
    # Check input directory (if not exist -> error)
    if not os.path.exists(os.path.join(data_path, args.input_dir)):
        print "Error: input directory not exist"
        return False
    for data_type in ["train", "test"]:
        path = os.path.join(data_path, args.input_dir, data_type)
        if not (os.path.exists(path)):
            print "Error: input directory missing train or test directories"
            return False
    
    # Check output directory (if not exist -> create one)
    if not os.path.exists(os.path.join(data_path, args.output_dir)):
        os.makedirs(os.path.join(data_path, args.output_dir))
    for data_type in ["train", "test"]:
        path = os.path.join(data_path, args.output_dir, data_type)
        if not os.path.exists(path):
            os.makedirs(path)
    
    return True

# Function for extracting features with openSMILE (Return whether successed)
def extract_features(args):
    if not check_dirs(args):
        return False   # failed to read inputs
            
    # Iterate over wav audio files in input directory
    for data_type in ["train", "test"]:
        path_in = os.path.join(data_path, args.input_dir, data_type)
        count = 0
        for filename in os.listdir(path_in):
            # For wav files
            if filename.endswith(".wav"):
                # in
                file_in = os.path.join(path_in, filename)
                config = os.path.join(data_path, "config", args.config)
                
                filename = filename.strip(".wav")
                
                # out
                path_out = os.path.join(data_path, args.output_dir, data_type)
                csv_out = os.path.join(path_out, filename + "_" + args.config.strip(".conf") + ".csv")
                arff_out = os.path.join(path_out, filename + "_" + args.config.strip(".conf") + ".arff")
                label = get_label(os.path.join(data_path, args.label), filename)
                
                # use openSMILE
                call(["SMILExtract", "-l", "0", "-noconsoleoutput", "-I", file_in, 
                      "-C", config, "-D", csv_out, "-O", arff_out, "-instname", filename, "-label", label])
                
                count += 1
                print ("Done: %d" % count)
                
    return True

# Obtaining args from terminal
def get_args():
    
    parser = argparse.ArgumentParser(description='Extract features for files in the directory using openSMILE')
    
    parser.add_argument("-i",                   # optional argument (no "-" for positional)
                        "--input_dir",   # name of the attribute (dest)
                        action = "store",       # can be "store", "store_const", "store_true", etc.
                        # nargs = N for associating N args with a single action
                        # const = ... to hold constant values
                        # default = ... to set default value
                        type = str,             # check arg type
                        # choice = [.., .., ..] # restrict set of values
                        required = True,        # make an option required
                        # metavar = "XXX" for changing display name
                        help = "The directory of input audio files (wav)")
    
    parser.add_argument("-o", "--output_dir", type = str, required = True, help = "The directory of results")
    parser.add_argument("-c", "--config", type = str, required = True, help = "Configuration filename")
    parser.add_argument("-l", "--label", type = str, required = True, help = "Label filename")
    args = parser.parse_args()
    
    return args

def main():
    # Obtaining terminal args
    args = get_args()
    
    # Extracting features according to args
    if not extract_features(args):
        print "Failed to extract features"
    else:
        print "Successfully extracted features"

# If running the file directly
if __name__ == "__main__":
    main()

Overwriting feature.py


Running script for extracting features. 

In [ ]:
%%!
python feature.py -i "input" -o "output" -c "IS09_emotion.conf" -l "label.csv"

## 3. Deep Neural Network for emotion recognition

Import the required modules.

In [24]:
from keras.models import Sequential

ImportError: cannot import name np_utils

In [15]:
model = Sequential()

NameError: name 'Sequential' is not defined